# Tensors

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from materialite import (
    Material,
    Orientation,
    Scalar,
    Order2Tensor,
    Order2SymmetricTensor,
    Order4SymmetricTensor,
    Vector,
    SlipSystem,
)

`Tensor`s in Materialite are intended to simplify tensor operations, which often require careful tracking of array dimensions and special handling depending on whether the array represents a single tensor or a group of tensors (e.g., at all the points in a material). Specific classes are:
* `Scalar`
* `Vector`
* `Order2Tensor`: non-symmetric second-order tensors
* `Order2SymmetricTensor`: symmetric second-order tensors. 
* `Order4SymmetricTensor`: fourth-order tensors with major and minor symmetries (e.g., stiffness tensors)
* `Orientation`: represents basis transformations

This demo walks through tensor operations, including individual tensors and groups of tensors, which are all handled by the above classes.

Create a displacement gradient for simple shear

In [ ]:
displacement_gradient_array = np.array([[0, 1.0, 0], [0, 0, 0], [0, 0, 0]])
displacement_gradient = Order2Tensor(displacement_gradient_array)
displacement_gradient

Recover the numpy array by asking for the tensor components

In [ ]:
displacement_gradient.components

The infinitesimal strain tensor is the symmetric part of the displacement gradient

In [ ]:
strain = displacement_gradient.sym
strain

The default representation of an `Order2SymmetricTensor` is the Mandel components. We can extract the Cartesian components instead if we want.

In [ ]:
strain.cartesian

...or the Voigt components. Note that we need to specify `strain_voigt` to get the correct representation (i.e., shear components multiplied by two)

In [ ]:
strain.strain_voigt

`stress_voigt` returns the stress Voigt representation (shear components not multiplied by two)

In [ ]:
strain.stress_voigt

We can construct an `Order2SymmetricTensor` from these different component representations too.

In [ ]:
strain2 = Order2SymmetricTensor(strain.components)
strain3 = Order2SymmetricTensor.from_mandel(strain.components)
strain4 = Order2SymmetricTensor.from_strain_voigt(strain.strain_voigt)
strain5 = Order2SymmetricTensor.from_stress_voigt(strain.stress_voigt)
strain6 = Order2SymmetricTensor.from_cartesian(strain.cartesian)
print(np.allclose(strain.components, strain2.components))
print(np.allclose(strain.components, strain3.components))
print(np.allclose(strain.components, strain4.components))
print(np.allclose(strain.components, strain5.components))
print(np.allclose(strain.components, strain6.components))

Note that `from_cartesian` raises an error if the provided numpy array is not symmetric

In [ ]:
try:
    _ = Order2SymmetricTensor.from_cartesian(displacement_gradient_array)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

For solid mechanics problems, we need stiffness tensors too. These are `Order4SymmetricTensor`s in Materialite. Here, we'll create a stiffness tensor for a material with cubic symmetry.

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(
    C11=250000, C12=140000, C44=100000
)
stiffness_tensor

Again, the default representation is the Mandel components. We can also ask for the Voigt components.

In [ ]:
stiffness_tensor.voigt

We can also create stiffness tensors for other materials:
* Isotropic: `Order4SymmetricTensor.from_isotropic_constants(modulus, shear_modulus)`
* Transversely isotropic (e.g., hexagonal close-packed materials): `Order4SymmetricTensor.from_transverse_isotropic_constants(C11, C12, C13, C33, C44)`
* Any stiffness tensor where you have the Voigt components in a Numpy array: `Order4SymmetricTensor.from_voigt(voigt_stiffness_matrix)`

Now let's do some tensor operations. First, get the stress from the stiffness tensor and the strain: $\mathbf{\sigma}$ = $\mathbb{C} \mathbf{\varepsilon}$ or, in index notation, $\sigma_{ij} = C_{ijkl} \varepsilon_{kl}$. Tensor applications use the `@` operator.

In [ ]:
stress = stiffness_tensor @ strain
stress

We can also do inner products like $\mathbf{\sigma} : \mathbf{\varepsilon}$ or, in index notation, $\sigma_{ij} \varepsilon_{ij}$, using the `*` operator. This returns a `Scalar`.

In [ ]:
stress * strain

Similarly, we can apply `Order2SymmetricTensor`s or `Order2Tensor`s to `Vector`s. Here, determine the traction vector on a particular surface:

In [ ]:
surface_normal = Vector([1.0, 1.0, 1.0]).unit
traction = stress @ surface_normal
traction

Note that `.unit` converted the `surface_normal` vector to a unit vector:

In [ ]:
surface_normal

`Tensor`s are most useful for doing tensor operations when you have multidimensional data. In Materialite, the dimensions are the points of the material and the slip systems that are available at each point. First, set up tensors with a points dimension.

In [ ]:
displacement_gradients_array = (
    np.array(
        [displacement_gradient_array, np.array([[-0.3, 0, 0], [0, -0.3, 0], [0, 0, 1]])]
    )
    * 0.001
)
displacement_gradients = Order2Tensor(displacement_gradients_array)
displacement_gradients

Operations we looked at previously remain the same.

In [ ]:
strains = displacement_gradients.sym
strains

In [ ]:
stresses = stiffness_tensor @ strains
stresses

Note that no special broadcasting operations were needed to make the `stiffness_tensor` (which has no points dimension) work correctly with `strains` (which has a points dimension).

In [ ]:
stresses * strains

We can do other useful operations like extracting the deviatoric part of a tensor or taking the norm.

In [ ]:
stresses.dev

In [ ]:
stresses.dev.norm  # note that this returns a Scalar with a points dimension

Here is the von Mises stress

In [ ]:
np.sqrt(3 / 2) * stresses.dev.norm

We can also take the mean over all the points. Note that `mean()` requires parenthesis.

In [ ]:
stresses.mean()

For crystal plasticity operations, we will also need to work with a slip systems dimension. This typically comes from the Schmid tensor. In Materialite, the `SlipSystem` class can automatically create the Schmid tensor for common slip system families.

In [ ]:
slip_systems = SlipSystem.octahedral()
schmid_tensor = slip_systems.schmid_tensor.sym
schmid_tensor

Note that the Schmid tensor has a "slip systems" dimension.

Now, suppose our two points from earlier are in a crystal with octahedral slip systems. We can easily compute the resolved shear stress on each slip system at each point. Note that we have two points and 12 slip systems.

In [ ]:
resolved_shear_stresses = schmid_tensor * stresses
resolved_shear_stresses

Now let's set up a viscoplastic constitutive law and compute the slip rates on each slip system from $\dot{\gamma}_s = \dot{\gamma}_0 \left(\frac{\tau_s}{g_s} \right)^{m-1} \left(\frac{\tau_s}{g_s} \right)$. Numbers that stay constant can just be regular numbers, but we'll make the slip resistance a `Scalar` so that it can be updated. Note that all of the slip rates on every slip system at every point are computed at once.

In [ ]:
hardening_rate = 10.0
reference_slip_rate = 1.0
slip_resistance = Scalar(100.0)
rate_exponent = 10.0
plastic_slip_rates = (
    reference_slip_rate
    * (resolved_shear_stresses / slip_resistance).abs ** (rate_exponent - 1)
    * (resolved_shear_stresses / slip_resistance)
)
plastic_slip_rates

Now we can compute the plastic strain rates at each point: $\dot{\mathbf{\varepsilon}}_p = \sum_s \mathbf{M}_s \dot{\gamma}_s$. Recall that the Schmid tensor is an `Order2SymmetricTensor` with a "points" dimension and the plastic slip rates are a `Scalar` with a "points" and a "slip systems" dimension. Therefore, the product will be an `Order2SymmetricTensor` with "points" and "slip systems" dimensions.

In [ ]:
plastic_strain_rates = (schmid_tensor * plastic_slip_rates).sum()
plastic_strain_rates

Note that the `sum` method automatically sums over the "slip systems" dimension if it exists. This behavior can also be specified explicitly.

In [ ]:
(schmid_tensor * plastic_slip_rates).sum("s")

Finally, update the slip resistances using a simple linear hardening law with the accumulated slip at each point: $g_s = g_{s,\text{old}} + H \Delta t \sum_s |\dot{\gamma}_s|$

In [ ]:
delta_t = 0.001
new_slip_resistances = (
    slip_resistance + hardening_rate * plastic_slip_rates.abs.sum() * delta_t
)
new_slip_resistances

Note that `new_slip_resistances` has a points dimension since the accumulated slip was calculated at each point. In other words, at each point, we have computed the new CRSS across all 12 slip systems. `new_slip_resistances` would also have a slip systems dimension if the hardening law computed different CRSS increments for each slip system. We also used the `abs` property of a `Scalar` to take the absolute value of the slip rates.

Generally, points will belong to different crystals that may have different orientations. All `Tensor`s (except `Scalar`s) are equipped with `to_crystal_frame()` and `to_specimen_frame()` methods to handle coordinate transformations. The only argument to the method is an `Orientation` object, which can contain a single orientation or a group of orientations just like `Tensor`s. `Orientation`s represent a basis transformation between a specimen reference frame and a crystal reference frame. There is more detail on creating and using `Orientation`s in [Crystallography](crystallography).

In [ ]:
rng = np.random.default_rng(12345)
orientations = Orientation.random(num=2, rng=rng)
specimen_frame_stiffnesses = stiffness_tensor.to_specimen_frame(orientations)
specimen_frame_stiffnesses

Now, each point has a different stiffness tensor in the specimen frame due to the different orientations. We also need to transform the Schmid tensor to the specimen frame.

In [ ]:
specimen_frame_schmid_tensors = schmid_tensor.to_specimen_frame(orientations)
specimen_frame_schmid_tensors

The constitutive law calculation can now be carried out exactly as before.

Other things...

The user can specify whether they want a `Tensor` to have a "points" or "slip systems" dimension. Materialite assumes there is a "points" dimension unless the user specifies otherwise.

In [ ]:
print("tensor with points dimension:")
print(Order2Tensor(displacement_gradients_array))
print("tensor with slip systems dimension:")
print(Order2Tensor(displacement_gradients_array, "s"))

If the user passes a single tensor to the constructor but asks for a "points" or "slip systems" dimension, Materialite raises an error.

In [ ]:
try:
    _ = Order2Tensor(displacement_gradient_array, "p")
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

`mean()` and `sum()` (for all `Tensor`s) and `max()` (for `Scalars`) automatically operate over the "slip systems" dimension if it exists. Otherwise, they operate over the "points" dimension. The user can also specify which dimension to act on.

In [ ]:
print(plastic_slip_rates.max())
print(plastic_slip_rates.max("s"))
print(plastic_slip_rates.max("p"))

Tensors can be added as fields to a material

In [ ]:
material = Material(dimensions=[2, 2, 2])
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(
    C11=250000, C12=140000, C44=100000
)
orientations = Orientation.random(num=material.num_points)
strain = (
    Order2SymmetricTensor.from_cartesian([[1.0, 0, 0], [0, 0, 0], [0, 0, 0]]) * 0.001
)
stresses = stiffness_tensor.to_specimen_frame(orientations) @ strain
material = material.create_fields({"orientation": orientations, "stress": stresses})
material.get_fields()

In [ ]:
extracted_stresses = material.extract("stress")
np.allclose(stresses.components, extracted_stresses.components)

You can index `Tensor`s like numpy arrays.

In [ ]:
specimen_frame_schmid_tensors[0]

In [ ]:
specimen_frame_schmid_tensors[0, 1]

In [ ]:
specimen_frame_schmid_tensors[:, 1]

However, you *cannot* pull off particular tensor components or an error will be raised.

In [ ]:
try:
    specimen_frame_schmid_tensors[0, 1, 0]
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")

In [ ]:
specimen_frame_schmid_tensors.components[0, 1, 0]

You can also set elements of a tensor with another tensor of the same type

In [ ]:
new_stiffnesses = specimen_frame_stiffnesses.copy()
new_stiffnesses[1] = specimen_frame_stiffnesses[0]
new_stiffnesses

Like numpy arrays, creating an instance of a `Tensor` with another tensor returns a copy of that tensor.

In [ ]:
vector1 = Vector([[1, 2, 3], [4, 5, 6]])
vector2 = Vector(vector1)
np.array_equal(vector1.components, vector2.components)

Confirm that we made a copy of the original `Vector`.

In [ ]:
vector2[1] = vector1[0]
np.array_equal(vector1.components, vector2.components)